# v0.0.1 of colab-ML-AMR

Select species, bin_size, models and dataset.

Run all -> Choose files (zip file of MALDI-TOF MS spectra)

In [ ]:
#@title Input protein sequence(s), then hit `Runtime` -> `Run all`
from google.colab import files
import os
import re
import hashlib
import random
import glob

from sys import version_info
python_version = f"{version_info.major}.{version_info.minor}"

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

species = "Staphylococcus_aureus" #@param ['Escherichia_coli', 'Klebsiella_pneumoniae','Staphylococcus_aureus'] {type:"string"}
#@markdown  - Use `_` to concat genus_species name
bin_size = 3 #@param [3] {type:"raw"}
#@markdown - (Da) 3 means that 6,000 bins of 2,000~20,000 Da of spectra range
models = 'All' #@param ['All', 'LogReg'] {type:"raw"}
#@markdown - specify pretrained models
dataset = 'UMG' #@param ['UMG'] {type:"raw"}
#@markdown - specify pretrained models
#jobname = 'test' #@param {type:"string"}

"""
# remove whitespaces
#basejobname = "".join(jobname.split())
#basejobname = re.sub(r'\W+', '', basejobname)
#jobname = add_hash(basejobname, species)

# check if directory with jobname exists
def check(folder):
  if os.path.exists(folder):
    return False
  else:
    return True
if not check(jobname):
  n = 0
  while not check(f"{jobname}_{n}"): n += 1
  jobname = f"{jobname}_{n}"
"""

!rm -rf ./data/* 
# make directory to save results
data_store_path = os.path.join('data', 'raw', species)
os.makedirs(data_store_path, exist_ok=True)

# Number of bins calculate
number_of_bins = int(round(18000/bin_size))

uploaded = files.upload()
use_templates = True
for fn in uploaded.keys():
  new_fname = os.path.join(data_store_path,fn)
  os.rename(fn, new_fname)
  if '.zip' in fn:
    import zipfile
    with zipfile.ZipFile(new_fname, 'r') as zip_ref:
        zip_ref.extractall(data_store_path)
    os.remove(new_fname)
  elif '.tar.gz' in fn:
    import tarfile
    if new_fname.endswith("tar.gz"):
        tar = tarfile.open(new_fname, "r:gz")
        tar.extractall()
        tar.close()
    elif new_fname.endswith("tar"):
        tar = tarfile.open(new_fname, "r:")
        tar.extractall()
        tar.close()
    os.remove(new_fname)

# Dataset filter
dataset_convert = {
    "All": '*',
    "UMG": 'UMG-0',
    "DRIAMS-A": 'DRIAMS-A',
}
dataset_search = dataset_convert[dataset]

# Model filter
model_convert = {
    "All": '*',
    "LogReg": 'lr',
    "LightGBM": 'LightGBM',
    "MLP": 'MLP',
}
model_search = model_convert[models]

# model download
model_store_path = os.path.join('models', 'sklearn')
os.makedirs(model_store_path, exist_ok=True)

model_download_check = os.path.join(model_store_path, species, f'Train_site_{dataset_search}_Model_{model_search}_Species_{species}*')
if len(glob.glob(model_download_check)) == 0:
  !gdown 1Atbf3y7xM58ZIbRXVAnvDPCT2aR_t_tJ

  # unzip model
  import glob
  model_file = 'sklearn_13092024.zip'
  nfilename = os.path.join(model_store_path, model_file)
  os.rename(model_file, nfilename)
  with zipfile.ZipFile(nfilename, 'r') as zip_ref:
      zip_ref.extractall(model_store_path)
else:
  print('models are exist!')
#
print("dataset", dataset_search)
print("model", model_search)
print("species",species)
print("bin size:",bin_size)

### Install dependencies

In [ ]:
!python -m pip install rpy2==3.5 pandas numpy

In [ ]:
# import rpy2's package module
import rpy2.robjects.packages as rpackages

# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list

# R package names
packnames = ('readBrukerFlexData', 'MALDIquant')

# R vector of strings
from rpy2.robjects.vectors import StrVector

# Selectively install what needs to be install.
# We are fancy, just because we can.
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))

### Spectra preprocessing

In [ ]:
import rpy2.robjects as robjects

robjects.r('''
require("readBrukerFlexData")
require("MALDIquant")
readBruker <- function(path_, out_path_) {
    sample <- readBrukerFlexDir(path_, removeCalibrationScans = TRUE,
                                removeMetaData = FALSE, useHpc = TRUE, useSpectraNames = TRUE,
                                filterZeroIntensities = FALSE, verbose = FALSE)

    m <- sample[[1]][[1]]$mass
    i <- sample[[1]][[1]]$intensity

    #Basic Preprocessing
    # (In this version the data was NOT transformed and normalized. That will hapen later in python)
    spectra <- createMassSpectrum(mass = m, intensity = i)
    spectra <- transformIntensity(spectra, method="sqrt")
    spectra <- smoothIntensity(spectra, method="SavitzkyGolay", halfWindowSize=10)
    spectra <- removeBaseline(spectra, method="SNIP", iterations=20)
    spectra <- calibrateIntensity(spectra, method="TIC")
    spectra <- trim(spectra, range=c(2000, 20000))

    #pks <- detectPeaks(spectra, method="MAD", halfWindowSize=20, SNR=4)
    pks <- spectra

    mass <- mass(pks)
    intensity <- intensity(pks)
    small_dataframe <- data.frame(mass, intensity, stringsAsFactors = FALSE)

    write.table(small_dataframe, out_path_, row.names = F, col.names = F)
    }
    ''')
read_bruker = robjects.globalenv['readBruker']

In [ ]:
import glob
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

def folder_scan(raw_dir: str) -> dict:
    file_exist_dic = {}
    raw_file_path = os.path.join(raw_dir, '*', '*')
    raw_file_list = glob.glob(raw_file_path)
    for filepath in raw_file_list:
        species_name, sample_number = filepath.split(os.sep)[-2:]
        if species_name not in file_exist_dic.keys():
            file_exist_dic[species_name] = set()
        file_exist_dic[species_name].add(sample_number)
    print(f'File scan done.')

    return file_exist_dic


def preprocessing(raw_dir: str, preprocessed_dir: str, file_exist_dic: dict) -> None:
    for species in file_exist_dic.keys():
        raw_path_species = os.path.join(raw_dir, species)
        preprocessed_path = os.path.join(preprocessed_dir, species)
        os.makedirs(preprocessed_path, exist_ok=True)
        for sample_number in file_exist_dic[species]:
            raw_path = os.path.join(raw_path_species, sample_number)
            preprocessed_filepath = os.path.join(preprocessed_path, sample_number)
            preprocessed_filepath = f'{preprocessed_filepath}.txt'

            if os.path.exists(preprocessed_filepath):
                #print(f'Preprocessing {preprocessed_filepath} already exist.')
                continue

            print(f'New raw file: {raw_path} found.')
            read_bruker(raw_path, preprocessed_filepath)

            try:
                read_bruker(raw_path, preprocessed_filepath)
                print(f'Preprocessing {preprocessed_filepath} done.')
            except:
                print(f'Preprocessing of {raw_path} fail.')

    return


def bin_vectorize(preprocessed_file: str, binned_file: str, bin_size: int) -> None:
    spectra = pd.read_csv(preprocessed_file, sep=' ', index_col=False, header=None).to_numpy()
    combined_times = spectra[:, 0]
    min_range = min(2000, np.min(combined_times))
    max_range = max(20000, np.max(combined_times))

    _, bin_edges_ = np.histogram(combined_times, bin_size, range=(min_range, max_range))

    times = spectra[:, 0]
    indices = np.digitize(times, bin_edges_, right=True)

    valid = (indices >= 1) & (indices <= bin_size)
    spectrum = spectra[valid]

    # Need to update indices to ensure that the first bin is at
    # position zero.
    indices = indices[valid] - 1
    identity = np.eye(bin_size)

    vec = np.sum(identity[indices] * spectra[:, 1][:, np.newaxis], axis=0)
    np.savetxt(binned_file, vec, delimiter=",")

    return


def binning(preprocessed_dir: str, binned_dir: str, file_todo_dic: dict, bin_size: int) -> None:
    for species in file_todo_dic.keys():
        preprocessed_path_species = os.path.join(preprocessed_dir, species)
        binned_path = os.path.join(binned_dir, species)
        os.makedirs(binned_path, exist_ok=True)
        for sample_number in file_todo_dic[species]:
            preprocessed_file_path = os.path.join(preprocessed_path_species, sample_number)
            sample_outpath = os.path.join(binned_path, sample_number)

            if os.path.exists(sample_outpath):
                print(f'Preprocessing {sample_outpath} already exist.')
                continue

            print(f'New preprocessed file: {preprocessed_file_path} found.')
            bin_vectorize(preprocessed_file_path, sample_outpath, bin_size)

            try:
                bin_vectorize(preprocessed_file_path, sample_outpath, bin_size)
                print(f'Binning {sample_outpath} done.')
            except:
                print(f'Binning of {preprocessed_file_path} fail.')



def scan_preprocessing(bin_size: int) -> None:
    raw_dir = os.path.join('.', 'data', 'raw')
    preprocessed_dir = os.path.join('.', 'data', 'preprocessed')
    binned_dir = os.path.join('.', 'data', f'binned_{str(bin_size)}')
    os.makedirs(raw_dir, exist_ok=True)
    os.makedirs(preprocessed_dir, exist_ok=True)
    os.makedirs(binned_dir, exist_ok=True)

    file_exist_dic = folder_scan(raw_dir)
    print(file_exist_dic)
    preprocessing(raw_dir, preprocessed_dir, file_exist_dic)

    preprocessed_exist_dic = folder_scan(preprocessed_dir)
    print(preprocessed_exist_dic)
    _ = binning(preprocessed_dir, binned_dir, preprocessed_exist_dic, bin_size)

In [ ]:
scan_preprocessing(number_of_bins)

### ML model run

In [ ]:
import joblib

model_dir = os.path.join('models','sklearn')

all_models = glob.glob(f'{model_dir}/{species}/*{dataset_search}*{model_search}*.joblib')
print(all_models)
model_dic = []
for x in all_models:
    to = os.path.basename(x).split('_')
    meta_dic = {
        'train_site': [to[2]],
        'ML': [to[7]],
        'species': ['_'.join(to[9:10])],
        'antibiotics': [to[12]],
        'seed': [to[14]],
        'model': joblib.load(x)
    }

    model_dic.append(meta_dic)
len(model_dic)

In [ ]:
def ML_model_run(model_dic: dict, bin_filepath: str, species: str) -> pd.DataFrame:
    vec = pd.read_csv(bin_filepath, sep=' ', index_col=False, header=None).to_numpy()
    vec = vec.T

    result = pd.DataFrame()
    list_of_models = model_dic
    for model_item in list_of_models:
        model = model_item['model']
        meta_data = model_item.copy()
        del meta_data['model']
        result_row = pd.DataFrame.from_dict(meta_data)
        pred = model.predict_proba(vec)[0]
        result_row['S'] = pred[0]
        result_row['R'] = pred[1]
        result = pd.concat([result, result_row], axis = 0)

    return result


def folder_scan(raw_dir: str) -> dict:
    file_exist_dic = {}
    raw_file_path = os.path.join(raw_dir, '*', '*')
    raw_file_list = glob.glob(raw_file_path)
    for filepath in raw_file_list:
        species_name, sample_number = filepath.split(os.sep)[-2:]
        if species_name not in file_exist_dic.keys():
            file_exist_dic[species_name] = set()
        file_exist_dic[species_name].add(sample_number)
    print(f'File scan done.')

    return file_exist_dic


def preprocessing(input_dir: str, output_dir: str, file_exist_dic: dict) -> None:
    pred_res = pd.DataFrame()
    for species in file_exist_dic.keys():
        raw_path_species = os.path.join(input_dir, species)
        preprocessed_path = os.path.join(output_dir, species)
        os.makedirs(preprocessed_path, exist_ok=True)
        for sample_number in file_exist_dic[species]:
            raw_path = os.path.join(raw_path_species, sample_number)
            preprocessed_filepath = os.path.join(preprocessed_path, sample_number)

            if os.path.exists(preprocessed_filepath):
                #print(f'Preprocessing {preprocessed_filepath} already exist.')
                continue

            print(f'New bin file: {raw_path} found.')
            pred_res = ML_model_run(model_dic, raw_path, species)

            try:
                pred_res = ML_model_run(model_dic, raw_path, species)
                print(f'ML prediction {preprocessed_filepath} done.')
            except:
                print(f'ML prediction of {raw_path} fail.')

    pred_res.sort_values('antibiotics', inplace=True)
    out_dic = []
    summary_res = pred_res.groupby('antibiotics')['S'].apply(list)
    for i_ in range(len(summary_res)):
        row = summary_res.values[i_]
        amname = summary_res.index[i_]
        row = ['S' if x > 0.5 else 'R' for x in row]
        out_dic.append({
            'Antibiotics': amname,
            'Resistant' : row.count('R'),
            'Susceptible': row.count('S')
        })
        #print(f"{name} S: {row.count('S')}, R: {row.count('R')}")

    result_df = pd.DataFrame.from_dict(out_dic)
    result_df.set_index(result_df['Antibiotics'], inplace=True)
    del(result_df['Antibiotics'])

    return result_df


binned_dir = os.path.join('.', 'data', f'binned_{str(number_of_bins)}')
bin_files = folder_scan(binned_dir)
results_dir = os.path.join('.', 'results')


final_result = preprocessing(binned_dir, results_dir, bin_files)

### Output

In [ ]:
final_result